In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import xgboost as xgb
import joblib
import time

# -------------------------------------------------------------------
# 1. Load and Prepare Data (Same as your LSTM script)
# -------------------------------------------------------------------
df_multi = pd.read_csv('combined_dataset.csv', index_col='datetime', parse_dates=True)
df_multi = df_multi[['stage_m'] + [col for col in df_multi.columns if col != 'stage_m']]

print("Columns in order:", df_multi.columns.tolist())

split_date_train_end = '2018-04-21'
split_date_val_start = '2019-01-01'
train_data = df_multi.loc[df_multi.index < split_date_train_end]
val_data = df_multi.loc[df_multi.index >= split_date_val_start]

# Load the scaler that was fit on this data during LSTM training
# This ensures we use the exact same scaling for a fair comparison
scaler = joblib.load('multivariate_combined_scaler.joblib')
scaled_train_data = scaler.transform(train_data)
scaled_val_data = scaler.transform(val_data)

N_PAST = 7
N_FUTURE = 7
n_features = scaled_train_data.shape[1]

def create_sequences(data, n_past, n_future):
    X, y = [], []
    target_col_index = 0
    for i in range(n_past, len(data) - n_future + 1):
        X.append(data[i - n_past:i, :])
        y.append(data[i:i + n_future, target_col_index])
    return np.array(X), np.array(y)

# Create sequences just like for the LSTM
X_train_seq, y_train = create_sequences(scaled_train_data, N_PAST, N_FUTURE)
X_val_seq, y_val = create_sequences(scaled_val_data, N_PAST, N_FUTURE)

# -------------------------------------------------------------------
# 2. Reshape Data for MLP and XGBoost
# -------------------------------------------------------------------
# We flatten the sequence of past data into a single feature vector.
# Shape changes from (samples, 7, 6) to (samples, 42)
X_train_flat = X_train_seq.reshape(X_train_seq.shape[0], -1)
X_val_flat = X_val_seq.reshape(X_val_seq.shape[0], -1)

print(f"\nOriginal sequence shape: {X_train_seq.shape}")
print(f"Flattened shape for MLP/XGBoost: {X_train_flat.shape}")

# -------------------------------------------------------------------
# 3. Train and Evaluate the Simple MLP Model
# -------------------------------------------------------------------
print("\n--- Training Simple MLP Model ---")

# Define MLP architecture
inputs = Input(shape=(X_train_flat.shape[1],)) # Input shape is the number of flattened features
dense1 = Dense(128, activation='relu')(inputs)
dropout1 = Dropout(0.2)(dense1)
dense2 = Dense(64, activation='relu')(dropout1)
# The output layer has N_FUTURE neurons, one for each day to predict
outputs = Dense(N_FUTURE)(dense2) 

mlp_model = Model(inputs=inputs, outputs=outputs)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
mlp_model.compile(optimizer=optimizer, loss='mean_squared_error')
mlp_model.summary()

# Train the model
start_time = time.time()
mlp_history = mlp_model.fit(
    X_train_flat, y_train,
    epochs=150,
    batch_size=128,
    validation_data=(X_val_flat, y_val),
    callbacks=[EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)],
    verbose=0 # Set to 0 to keep output clean
)
mlp_training_time = time.time() - start_time
print(f"MLP training completed in {mlp_training_time:.2f} seconds.")

# Save the MLP model
mlp_model.save('best_baseline_mlp_model.keras')
print("MLP model saved as 'best_baseline_mlp_model.keras'")

# Predict and evaluate
mlp_predictions_scaled = mlp_model.predict(X_val_flat)

# -------------------------------------------------------------------
# 4. Train and Evaluate the XGBoost Model
# -------------------------------------------------------------------
print("\n--- Training XGBoost Model ---")

# Instantiate the XGBoost regressor for multi-output regression
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,          # Will stop early
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    early_stopping_rounds=15,   # Stop if validation loss doesn't improve for 15 rounds
    n_jobs=-1                   # Use all available CPU cores
)

# Train the model
start_time = time.time()
xgb_model.fit(
    X_train_flat, y_train,
    eval_set=[(X_val_flat, y_val)],
    verbose=False               # Set to False to keep output clean
)
xgb_training_time = time.time() - start_time
print(f"XGBoost training completed in {xgb_training_time:.2f} seconds.")

# Save the XGBoost model
joblib.dump(xgb_model, 'best_baseline_xgboost_model.joblib')
print("XGBoost model saved as 'best_baseline_xgboost_model.joblib'")

# Predict and evaluate
xgb_predictions_scaled = xgb_model.predict(X_val_flat)


# -------------------------------------------------------------------
# 5. Inverse Scale and Compare All Models
# -------------------------------------------------------------------

def evaluate_model(model_name, predictions_scaled, y_true_scaled):
    """A helper function to inverse scale and calculate metrics."""
    # Create a dummy array for inverse scaling
    dummy_preds = np.zeros((len(predictions_scaled.flatten()), n_features))
    dummy_preds[:, 0] = predictions_scaled.flatten()
    predictions_original = scaler.inverse_transform(dummy_preds)[:, 0].reshape(y_true_scaled.shape)
    
    dummy_true = np.zeros((len(y_true_scaled.flatten()), n_features))
    dummy_true[:, 0] = y_true_scaled.flatten()
    y_true_original = scaler.inverse_transform(dummy_true)[:, 0].reshape(y_true_scaled.shape)
    
    print(f"\n--- {model_name} Model Performance on Validation Set ---")
    for i in range(N_FUTURE):
        day = i + 1
        mae = mean_absolute_error(y_true_original[:, i], predictions_original[:, i])
        rmse = np.sqrt(mean_squared_error(y_true_original[:, i], predictions_original[:, i]))
        r2 = r2_score(y_true_original[:, i], predictions_original[:, i])
        print(f"Day {day} Ahead -> MAE: {mae:.4f} m, RMSE: {rmse:.4f} m, R²: {r2:.4f}")

# Evaluate our new models
evaluate_model("Simple MLP", mlp_predictions_scaled, y_val)
evaluate_model("XGBoost", xgb_predictions_scaled, y_val)

print("\n--- For Comparison: Your Tuned LSTM Results ---")
print("Day 1 Ahead -> MAE: 0.1326 m, RMSE: 0.2424 m, R²: 0.8982")
print("Day 2 Ahead -> MAE: 0.2431 m, RMSE: 0.4457 m, R²: 0.6553")
print("Day 3 Ahead -> MAE: 0.3086 m, RMSE: 0.5467 m, R²: 0.4807")
print("Day 4 Ahead -> MAE: 0.3417 m, RMSE: 0.5930 m, R²: 0.3884")
print("Day 5 Ahead -> MAE: 0.3606 m, RMSE: 0.6165 m, R²: 0.3385")
print("Day 6 Ahead -> MAE: 0.3742 m, RMSE: 0.6312 m, R²: 0.3060")
print("Day 7 Ahead -> MAE: 0.3839 m, RMSE: 0.6426 m, R²: 0.2798")

Columns in order: ['stage_m', 'discharge_cms', 'tavg', 'prcp', 'wspd', 'pres', 'rhum']

Original sequence shape: (8203, 7, 7)
Flattened shape for MLP/XGBoost: (8203, 49)

--- Training Simple MLP Model ---


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 49)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         6,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,111 (59.03 KB)

 Trainable params: 15,111 (59.03 KB)

 Non-trainable params: 0 (0.00 B)

MLP training completed in 16.14 seconds.
MLP model saved as 'best_baseline_mlp_model.keras'
73/73 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step

--- Training XGBoost Model ---
XGBoost training completed in 4.47 seconds.
XGBoost model saved as 'best_baseline_xgboost_model.joblib'

--- Simple MLP Model Performance on Validation Set ---
Day 1 Ahead -> MAE: 0.1504 m, RMSE: 0.2873 m, R²: 0.8570
Day 2 Ahead -> MAE: 0.2422 m, RMSE: 0.4543 m, R²: 0.6417
Day 3 Ahead -> MAE: 0.3049 m, RMSE: 0.5549 m, R²: 0.4650
Day 4 Ahead -> MAE: 0.3377 m, RMSE: 0.5991 m, R²: 0.3756
Day 5 Ahead -> MAE: 0.3571 m, RMSE: 0.6250 m, R²: 0.3201
Day 6 Ahead -> MAE: 0.3814 m, RMSE: 0.6465 m, R²: 0.2722
Day 7 Ahead -> MAE: 0.3884 m, RMSE: 0.6550 m, R²: 0.2519

--- XGBoost Model Performance on Validation Set ---
Day 1 Ahead -> MAE: 0.1152 m, RMSE: 0.2362 m, R²: 0.9033
Day 2 Ahead -> MAE: 0.2333 m, RMSE: 0.4482 m, R²: 0.6514
Day 3 Ahead -> MAE: 0.3034 m, RMSE: 0.5538 m, R²: 0.4671
Day 4 Ahead -> MAE: 0.3377 m, RMSE: 0.5980 m, R²: 0.37